In [ ]:
pip install qiskit qiskit_ibm_runtime qiskit_aer


In [41]:
import qiskit

In [42]:
import qiskit
print(qiskit.__version__)  # 2.0.2

2.1.1


In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

# Paste your API key and CRN here
your_api_key = "paste_your_api_key_here"
your_crn = "paste_your_crn_here"   # CRN example: 'ibm-q/open/main' or your assigned CRN

# Save your IBM Quantum Platform account (new upgraded system)
QiskitRuntimeService.save_account(
    channel="ibm_quantum_platform",
    token=your_api_key,
    instance=your_crn,
    name="name_of_your_account",    # You can change this label if needed
    overwrite=True
)
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(name="name_of_your_account") # Use the name you set when saving your account
  # Avoids full backend listing
service.saved_accounts()


In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(name="name_of_your_account")  # Use the name you set when saving your account

# List available backends
backends = service.backends()
print("Available backends in your CRN:")
for backend in backends:
    print(backend.name)



In [ ]:
!pip install pylatexenc


In [ ]:
# Run a noisy biased Galton box simulation using Qiskit
# This code simulates a biased Galton box circuit with noise using Qiskit Aer and IBM Quantum backends.
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.visualization import circuit_drawer

import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import os

# Load IBM Quantum Platform account
service = QiskitRuntimeService(name="qgss-2025")
backend = service.backend('ibm_brisbane')  # Change backend if needed
noise_model = NoiseModel.from_backend(backend)

# Parameters
layers_list = [5, 10, 15, 20, 25]
theta = np.pi / 6
shots = 10000
output_folder = './results/noise'
os.makedirs(output_folder, exist_ok=True)

def run_biased_galton_box(n_layers, theta):
    qc = QuantumCircuit(n_layers, n_layers)
    for i in range(n_layers):
        qc.ry(2 * theta, i)
    qc.measure(range(n_layers), range(n_layers))
    return qc

def counts_to_probs(counts, n_layers):
    total = sum(counts.values())
    counts_right = Counter()
    for bitstring, freq in counts.items():
        num_right = bitstring.count('1')
        counts_right[num_right] += freq
    return np.array([counts_right.get(i, 0)/total for i in range(n_layers+1)])

for n_layers in layers_list:
    print(f"Running Task 4 for {n_layers} layers...")

    qc = run_biased_galton_box(n_layers, theta)

    ideal_sim = AerSimulator()
    noisy_sim = AerSimulator(noise_model=noise_model)

    qc_ideal = transpile(qc, ideal_sim)
    qc_noisy = transpile(qc, noisy_sim)

    result_ideal = ideal_sim.run(qc_ideal, shots=shots).result()
    result_noisy = noisy_sim.run(qc_noisy, shots=shots).result()

    counts_ideal = result_ideal.get_counts()
    counts_noisy = result_noisy.get_counts()

    probs_ideal = counts_to_probs(counts_ideal, n_layers)
    probs_noisy = counts_to_probs(counts_noisy, n_layers)
    tvd = 0.5 * np.sum(np.abs(probs_ideal - probs_noisy))

    fig, axs = plt.subplots(1, 3, figsize=(18, 5))

    # ✅ Replaced circuit drawer with ASCII embedding
    ascii_circuit = circuit_drawer(qc, output='text')
    axs[0].text(0.5, 0.5, ascii_circuit, fontsize=10, family='monospace',
                ha='center', va='center', wrap=True)
    axs[0].axis('off')
    axs[0].set_title(f'Circuit (ASCII): {n_layers} Layers, θ={round(theta, 2)}')

    x = np.arange(0, n_layers + 1)
    axs[1].bar(x - 0.2, probs_ideal, width=0.4, label='Ideal', color='skyblue')
    axs[1].bar(x + 0.2, probs_noisy, width=0.4, label='Noisy', color='salmon')
    axs[1].set_xlabel('Number of "Right" Moves (1\'s)')
    axs[1].set_ylabel('Probability')
    axs[1].set_title('Ideal vs Noisy')
    axs[1].legend()

    axs[2].bar(x, np.abs(probs_ideal - probs_noisy), color='gray')
    axs[2].set_xlabel('Number of "Right" Moves (1\'s)')
    axs[2].set_ylabel('Error')
    axs[2].set_title(f'Error per Bin\nTVD={round(tvd, 4)}')

    plt.tight_layout()
    filename = f"{output_folder}/noisy_biased_galton_box_{n_layers}_layers.png"
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()

    print(f"Saved: {filename}")


Running Task 4 for 5 layers...
Saved: ./results/noise/noisy_biased_galton_box_5_layers.png
Running Task 4 for 10 layers...
Saved: ./results/noise/noisy_biased_galton_box_10_layers.png
Running Task 4 for 15 layers...
Saved: ./results/noise/noisy_biased_galton_box_15_layers.png
Running Task 4 for 20 layers...
Saved: ./results/noise/noisy_biased_galton_box_20_layers.png
Running Task 4 for 25 layers...


/tmp/ipykernel_12381/4097517265.py:82: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()


Saved: ./results/noise/noisy_biased_galton_box_25_layers.png
